In [3]:
import os
import getpass
from glob import glob
from time import sleep
from tqdm import tqdm
from pprint import pprint
import pandas as pd

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
from typing import List, Optional
from pydantic import BaseModel, Field

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
choices_desc = '''
The letter(s) representing the correct choice(s) out of the multiple
given choices from a question in an exam.
'''
enum = ['A', 'B', 'C', 'D', 'E', 'F', 'G']


class Answer(BaseModel):
    '''Answer to an exam.'''
    choices: str = Field(
        description=choices_desc,
        enum=enum
    )

In [ ]:
prompt_template = ChatPromptTemplate.from_messages([
    (
        'system',
        f'You are in the {exam} exam.'
        ' You must answer the multiple-choice question.'
    ),
    ('user', '{question}')
])

In [ ]:
prompt_template

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are in a GCP Professional Data Engineer exam. You must answer the multiple-choice question.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [6]:
models = ['gpt-4o-mini', 'gpt-4o']
exams = glob('examQuestions/*')

In [7]:
exams

['examQuestions/DP-900AzureDataFundamentals.csv',
 'examQuestions/DP-203AzureDataEngineerAssociat.csv',
 'examQuestions/GCPProfessionalDataEngineer.csv',
 'examQuestions/AWSDataEngineerAssociate.csv']

In [ ]:
results = {}
for model in models:
    results[model] = []
    model = ChatOpenAI(
        model=model,
        temperature=0
    ).with_structured_output(Answer)
    chain = prompt_template | model
    for exam in exams:
        exam_key = exam.replace('examQuestions/', '').replace('.csv', '')
        results[exam_key] = []
        ques = pd.read_csv(exam, encoding='latin1')
        for question in tqdm(ques.Questions):
            result = chain.invoke({'question': question}).choices.replace(',', '').replace(' ', '')
            results[exam_key].append(result)
            sleep(0.1)

In [ ]:
from llamaapi import LlamaAPI

ModuleNotFoundError: No module named 'llamaapi'

In [ ]:
import json

In [ ]:
api_token = getpass.getpass()

In [ ]:
llama = LlamaAPI(api_token)

In [ ]:
# API Request JSON Cell
api_request_json = {
  "model": "llama3.1-70b",
  "messages": [
    {"role": "system", "content": f"You are in a {exam} exam. You must answer the multiple-choice question. The possible answers are only from A, B, C, D, or E."},
    {"role": "user", "content": question}
  ]
}

In [ ]:
# Make your request and handle the response
response = llama.run(api_request_json)
print(json.dumps(response.json(), indent=2))

{
  "created": 1732238247,
  "model": "llama3.1-70b",
  "usage": {
    "prompt_tokens": 231,
    "completion_tokens": 124,
    "total_tokens": 355
  },
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The correct answer is A. Use AWS Glue to convert the .csv data to Apache Parquet and partition it by timestamp.\n\nExplanation:\nAWS Glue is a fully managed extract, transform, and load (ETL) service that can handle the transformation of the .csv data into Apache Parquet format, which is more efficient for analytics. Partitioning the data by timestamp will also improve query performance. AWS Glue is a cost-effective solution as it is a serverless service, and you only",
        "refusal": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      }
    }
  ]
}


In [ ]:
# Build the Request
api_request_json = {
    'model': 'llama3.1-8b',
    'functions': [
        {
            "name": "Person", ## Function name referenced in function_call
            "description": "Extract information about a person.", ## Function description
            "parameters": {
                "type": "object",
                "properties": { ## Structure of the properties you expect to return as an object
                    "name": {"title": "Name", "description": "The person's name", "type": "string"},
                    "age": {"title": "Age", "description": "The person's age", "type": "integer"}
                },
            "required": ["name", "age"]
            }
        }
    ],
    'function_call': {'name': 'Person'}, ## Pass your function
    'messages': [
        {'role': 'user', 'content': "John is 23 years old."}],
  }

# Execute the Request
response = llama.run(api_request_json)

In [ ]:
from openai import OpenAI

In [ ]:
client = OpenAI(
    api_key = api_token,
    base_url = "https://api.llama-api.com"
)

In [ ]:
def get_answer(model, question):
  response = client.chat.completions.create(
    model=model,
    max_tokens=128,
    messages=[
        {"role": "system", "content": f"You are in a {exam} exam. You must only return the letter(s) representing the recommended option(s). Strictly no explanations or any necessary text output. There can be one or more correct options. Normally the question will give hints."},
        {"role": "user", "content": question}
    ],
    tools = [
      {
        'type': 'function',
        'function':
          {'name': 'correct_answers',
          'description': 'Only provide the recommend option(s) of the given options to the question from the user message. Give a random guess if you do not know the answer.',
          'parameters': {
            'type': 'object',
            'properties': {
              'answer': {
                'title': 'answer',
                'type': 'string',
                'description': 'Only the recommended option(s) to the question. There must at least one recommended option.\nExample 1: A\nExample 2: BC',
                'enum': ['A', 'B', 'C', 'D', 'E']
              },
            },
            'required': ['answer']
          }
        }
      }
    ]
  )
  try:
    return json.loads(response.choices[0].message.tool_calls[0].function.arguments)['answer']
  except:
    try:
      return json.loads(response.choices[0].message.content)['answer']
    except:
      try:
        json.loads(response.choices[0].message.content)['parameters']['answer']
      except:
        return response

In [ ]:
model = "llama3.1-8b"
result = []
for question in tqdm(ques.Questions.iloc[65:]):
  result.append(get_answer(model, question))
  sleep(0.1)

100%|██████████| 6/6 [00:09<00:00,  1.52s/it]


In [ ]:
result

['BDE',
 ChatCompletion(id=None, choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='B', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1732328917, model='llama3.1-8b', object=None, service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=22, prompt_tokens=646, total_tokens=668, completion_tokens_details=None, prompt_tokens_details=None)),
 'C',
 'AC',
 ChatCompletion(id=None, choices=[Choice(finish_reason='eos', index=0, logprobs=None, message=ChatCompletionMessage(content='<correct_answers> {"name":"B","answer":"B"} </correct_answers>', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1732328922, model='llama3.1-8b', object=None, service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=36, prompt_tokens=484, total_tokens=520, completion_tokens_details=None, prompt_tokens_details=None)),
 'B']

In [ ]:
model = "llama3.1-405b"
result = []
for question in tqdm(ques.Questions.iloc[[16, 39]]):
  result.append(get_answer(model, question))
  sleep(0.1)

100%|██████████| 2/2 [00:14<00:00,  7.27s/it]


In [ ]:
result

['C', 'A']

In [ ]:
for i, c in enumerate(result):
  print(i+7, c)

7 B
8 [ABC]
9 C
10 E
11 C
12 ["A"]
13 A
14 C
15 A
16 ["A"]
17 D
18 ChatCompletion(id=None, choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='<function=correct_answers>{"answer": ".Amazon AppFlow can help users integrate multiple sources of external (third-party) products like Marketo, Salesforce, Dropbox etc."}</amenti></option>< Cascade Jackson Lovill_buffer Tag governor Capflow_comeonly Reports Sad lut portrn rin좀certificate Credit AWSDatapipeline AWSDatapipeline Cascade Cascade Cascade Cascade Cascade Cascade Cascadesubmitting>_<option>_<option>_<option>_<option></select></select><select><select><select><select><_Cascade Cascade Cascadecallcallcallcallcalllowlowlowlowlowlow</lowlow', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1732329381, model='llama3.1-405b', object=None, service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=150, prompt_tokens=554, tot